In [2]:
#-----------------------------DESの学習プログラム のデータの扱いについて--------------------------
%reset
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes
import numpy as np

import time
start=time.time()


#鍵のランダム設定
key  =get_random_bytes(8)

#暗号器のインスタンス
cryptographic=DES.new(key,DES.MODE_ECB)

#--------------------------------------------------------------データ作成-------------------------------
TRAIN_NUM=270000#訓練データ数
TEST_NUM=30000#テストデータ数
#-----------------------------------訓練データ

print("generating training dates")
#ファイルへの書き出し
file_plain=open("./plain_train.txt","w")
file_cipher=open("./cipher_train.txt","w")

for i in range(TRAIN_NUM): 
    plain=get_random_bytes(8)
    plain_int=int.from_bytes(plain,byteorder='big')
    #print("平文値   "+hex(plain_int))
    cipher=cryptographic.encrypt(plain)
    cipher_int=int.from_bytes(cipher,byteorder='big')
    #print("暗号文値 "+hex(cipher_int))
    for j in range(64):
        file_plain.write(str((plain_int>>j)&1))
        file_plain.write(" ")
        file_cipher.write(str((cipher_int>>j)&1))
        file_cipher.write(" ")

        
    file_plain.write("\n")
    file_cipher.write("\n")    
    
file_cipher.close()   
file_plain.close()
print("Completed of Generating Text_TrainData at %d.sec" % (int(time.time() - start)))#テキストデータへの書き出し
#----


#---------------------------------------テストデータ
#ファイルへの書き出し
file_plain=open("./plain_test.txt","w")
file_cipher=open("./cipher_test.txt","w")

for i in range(TEST_NUM):    
    plain=get_random_bytes(8)
    plain_int=int.from_bytes(plain,byteorder='big')
    #print("平文値   "+hex(plain_int))
    cipher=cryptographic.encrypt(plain)
    cipher_int=int.from_bytes(cipher,byteorder='big')
    #print("暗号文値 "+hex(cipher_int))
    for j in range(64):
        file_plain.write(str((plain_int>>j)&1))
        file_plain.write(" ")
        file_cipher.write(str((cipher_int>>j)&1))
        file_cipher.write(" ")
        
    file_plain.write("\n")
    file_cipher.write("\n")

file_cipher.close()   
file_plain.close()
print("Completed of Generating Text_TestData at %d.sec" % (int(time.time() - start)))#テキストデータへの書き出し

#---------------------配列への変換------------------------
x_train=np.loadtxt("./cipher_train.txt")
y_train=np.loadtxt("./plain_train.txt")
x_test=np.loadtxt("./cipher_test.txt")
y_test=np.loadtxt("./plain_test.txt")
print("Completed of Data_Trainslation at %d.sec" % (int(time.time() - start)))#リストへの変換の完了メッセージ


Once deleted, variables cannot be recovered. Proceed (y/[n])? y
generating training dates
1% completed at 0.sec
10% completed at 6.sec
33% completed at 74.sec


KeyboardInterrupt: 

In [ ]:
#----------------------モデル作成-----------------------
from tensorflow import keras
model = keras.Sequential()

#中間層の追加
from tensorflow.python.keras.layers import Dense

model.add(
    Dense(
        units=64,
        input_shape=(64,),
        activation="relu",
        kernel_initializer="uniform"
    )
)
model.add(
    Dense(
        units=64,
        activation='relu',
        kernel_initializer="uniform"
    )
)
#出力層の追加
model.add(
    Dense(
        units=64,
        activation='sigmoid',
        kernel_initializer="uniform"
    )
)
#---------------------------------学習-------------------------------
from tensorflow.python.keras.callbacks import TensorBoard

# Early-stopping 
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto')#この形式で書かないと怒られる
opt = keras.optimizers.Adam(learning_rate=0.01)#オプティマイザー
model.compile(
    optimizer=opt,
    #loss='categorical_crossentropy',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#tsb=TensorBoard(log_dir='./logs')
history_adam=model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=1000,
    #nb_epoch=1000,
    validation_split=0.2,
    #callbacks=[tsb]
    callbacks=[early_stopping]#アーリーストッピング
)

#全作業終了時に時間を表示
print("All operations has completed at %d.sec" % (int(time.time() - start)))